# 【問題1】全結合層のクラス化
全結合層のクラス化を行なってください。


以下に雛形を載せました。コンストラクタで重みやバイアスの初期化をして、あとはフォワードとバックワードのメソッドを用意します。重みW、バイアスB、およびフォワード時の入力Xをインスタンス変数として保持しておくことで、煩雑な入出力は不要になります。


なお、インスタンスも引数として渡すことができます。そのため、初期化方法のインスタンスinitializerをコンストラクタで受け取れば、それにより初期化が行われます。渡すインスタンスを変えれば、初期化方法が変えられます。


また、引数として自身のインスタンスselfを渡すこともできます。これを利用してself.optimizer.update(self)という風に層の重みの更新が可能です。更新に必要な値は複数ありますが、全て全結合層が持つインスタンス変数にすることができます。


初期化方法と最適化手法のクラスについては後述します。



In [381]:
import numpy as np

In [382]:
class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.initializer = initializer
        self.optimizer = optimizer
        self.B = SimpleInitializer.B
        self.W = SimpleInitializer.W

        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        pass
    def forward(self, X, batch_size=20):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """        
        #self.batch_size = batch_size 
        A = X@self.w + self.b #(20, 784) (784, 400) (20, 400)
        return A
    
    def backward(self, dA, y=None):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """
        dZ = dA@self.w.T # (20, 784) (784, 400)  (20, 400)
        dW = dZ.T@dA
        dB = np.sum(dA, axis=0)
        
        # 更新
        self = self.optimizer.update(self)
        
        return dZ

In [383]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

In [384]:
get_mini_batch = GetMiniBatch(X_train, y_train, batch_size=20)
print(len(get_mini_batch)) # 2400
print(get_mini_batch[5]) # 5番目のミニバッチが取得できる
for mini_X_train, mini_y_train in get_mini_batch:
    pass

3000
(array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([4, 8, 4, 1, 3, 4, 9, 1, 3, 1, 6, 4, 9, 1, 8, 8, 8, 4, 3, 6],
      dtype=uint8))


# 【問題2】初期化方法のクラス化
初期化を行うコードをクラス化してください。


前述のように、全結合層のコンストラクタに初期化方法のインスタンスを渡せるようにします。以下の雛形に必要なコードを書き加えていってください。標準偏差の値（sigma）はコンストラクタで受け取るようにすることで、全結合層のクラス内にこの値（sigma）を渡さなくてすむようになります。


これまで扱ってきた初期化方法はSimpleInitializerクラスと名付けることにします。

In [385]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    --------------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
    
    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W :
        """
        self.W = np.random.randn(n_nodes1, n_nodes2) #例１層目 (784, 400)
        return self.W
    
    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :
        """
        self.B = np.random.randn(n_nodes2)
        return self.B

# 【問題3】最適化手法のクラス化
最適化手法のクラス化を行なってください。


最適化手法に関しても初期化方法同様に全結合層にインスタンスとして渡します。バックワードのときにself.optimizer.update(self)のように更新できるようにします。以下の雛形に必要なコードを書き加えていってください。


これまで扱ってきた最適化手法はSGDクラス（Stochastic Gradient Descent、確率的勾配降下法）として作成します。



In [386]:
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
        self.B = SimpleInitializer.B
        self.W = SimpleInitializer.W
        # ❻lrは親ノードに記載するもの？？
        # →感覚的にはそんな気がする
    
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        self.B -= lr * np.sum(layer, axis=0) # (400,)
        self.W -= lr * X.T@layer # (784, 20) (20,400) (784, 400)        

# 【問題4】活性化関数のクラス化
活性化関数のクラス化を行なってください。


ソフトマックス関数のバックプロパゲーションには交差エントロピー誤差の計算も含む実装を行うことで計算が簡略化されます。




In [387]:
# シグモイド関数
class Sigmoid:
    def sigmoid(self, X):
        s = 1/(1 + np.exp(-X))
        return s
    
    def backward(self, X):
        s_f = (1 - self.sigmoid(X)) * self.sigmoid(X) 
        return s_f

In [388]:
# ソフトマックス    
class SoftMax:
    def soft_max(self, X):
        # オーバーフロー対策 
        X_max = np.max(X) # (20,10)
        sft = np.exp(X - X_max)/ np.sum(np.exp(X - X_max), axis=1, keepdims=True)
        return sft
    
    def backward(self, X):
        stf_f = self.soft_max(X)*(1-self.soft_max(X))
        return sft_f
    
    def log_loss(self, X, y):
        self.loss = -(1/self.batch_size) * np.sum(y*np.log(self.forward(X)))
        return self.loss

In [389]:
# ハイパポリックタンジェント
class Tanh:
    def __init__(self):
        pass
    def tanh(self, X):
        t = np.tanh(X)
        return t
    
    def backward(self, X):
        t_f = 1 - self.tanh(X)**2
        return t_f

# 【問題5】ReLUクラスの作成
現在一般的に使われている活性化関数であるReLU（Rectified Linear Unit）をReLUクラスとして実装してください。


ReLUは以下の数式です。

In [390]:
class ReLU:
    def __init__(self):
        pass
    
    def relu(self, X):
        f = np.maximum(0, X)
        return f
    
    def backward(self, X):
        f_x = np.where(X>0, 1, 0)
        return f_x

# 【問題6】重みの初期値
ここまでは重みやバイアスの初期値は単純にガウス分布で、標準偏差をハイパーパラメータとして扱ってきました。しかし、どのような値にすると良いかが知られています。シグモイド関数やハイパボリックタンジェント関数のときは Xavierの初期値 （またはGlorotの初期値）、ReLUのときは Heの初期値 が使われます。


XavierInitializerクラスと、HeInitializerクラスを作成してください。

In [391]:
# sgmoid、tanhの初期の重み、バイアスはxavierが飽和しにくい
import numpy as np
class XavierInitializer:
    def __init__(self):
        pass
    
    def Xavier(nodes):
        sigma = 1/np.sqrt(nodes)
        return sigma

# reluの初期の重み、バイアスはheが飽和しにくい
class HeInitializer:
    def __init__(self):
        pass
    
    def he(self, nodes):
        sigma = np.sqrt(2/nodes)
        return sigma

a = XavierInitializer

# 【問題7】最適化手法
学習率は学習過程で変化させていく方法が一般的です。基本的な手法である AdaGrad のクラスを作成してください。


まず、これまで使ってきたSGDを確認します。





In [392]:
class AdaGlad:
    def __init__(self, lr):
        self.lr = lr
        pass
    
    def adaglad(self, dA):
        dZ = dA@self.w.T # (20, 784) (784, 400)  (20, 400)
        h_W +=  (dZ.T@dA)**2
        h_B += np.sum(dA, axis=0)**2
        self.W -= lr * 1/(np.sqrt(h_W)) * FC.backward(X)
        self.B -= lr * 1/(np.sqrt(h_B)) * FC.backward(X)

# 【問題8】クラスの完成
任意の構成で学習と推定が行えるScratchDeepNeuralNetrowkClassifierクラスを完成させてください。


In [749]:
# 親クラス
class ScratchDeepNeuralNetrowkClassifier:
    def __init__(self,n_features=784, n_nodes1=400, n_nodes2=200, n_output=10, batch_size=20, sigma=0.01, lr=0.01, epoch=20,optimizer="ada",activation = "tanh",inital = "xavier",verbose=False):
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.n_output = n_output
        self.n_features = n_features
        self.batch_size = batch_size
        self.sigma = sigma
        self.lr = lr
        self.epoch = epoch
        self.verbose = verbose
        self.optimizer = optimizer
        self.activation = activation
        self.inital = inital
        
    def fit(self, X, y, X_val=None, y_val=None):
        
        if self.optimizer == "ada":
            optimizer1 = AdaGlad(self.lr)
            optimizer2 = AdaGlad(self.lr)
            optimizer3 = AdaGlad(self.lr)
        elif self.optimizer == "sgd":
            optimizer1 = SGD(self.lr)
            optimizer2 = SGD(self.lr)
            optimizer3 = SGD(self.lr)
        
        
        if self.activation == "tanh":
            self.activation1 = Tanh()
            self.activation2 = Tanh()
            
        elif self.activation == "softmax":
            self.activation1 = SoftMax()
            self.activation2 = SoftMax()
        elif self.activation == "relu":
            self.activation1 = ReLU()
            self.activation2 = ReLU()        
        elif self.activation == "sigmoid":
            self.activation1 = Sigmoid()
            self.activation2 = Sigmoid()   
        
        if self.inital == "simple":
            initial = SimpleInitializer(self.sigma)
        elif self.inital == "xavier":
            initial = XavierInitializer()
        elif self.inital == "he":
            initial = He()
        
        # 3層のみsoftmax
        self.activation3 = Softmax(batch_size=self.batch_size)
        
        # 層の形成
        self.FC1 = FC(self.n_features, self.n_nodes1, initial, optimizer1) # for分の中だと初期化だめ
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, initial, optimizer2)
        self.FC3 = FC(self.n_nodes2, self.n_output, initial, optimizer3)
        
        # one-hot
        y_hot = y.reshape(-1, 1) == np.arange(10)
        y_val_hot = y_val.reshape(-1,1) == np.arange(10)
                
        # epoch
        for epoch in range(self.epoch):
            get_mini_batch = GetMiniBatch(X, y_hot, batch_size=20)
            train_loss = 0
            accuracy_scores = 0
            
            for mini_X_train, mini_y_train in get_mini_batch:
                A1 = self.FC1.forward(mini_X_train)
                Z1 = self.activation1.forward(A1)
                A2 = self.FC2.forward(Z1)
                Z2 = self.activation2.forward(A2)
                A3 = self.FC3.forward(Z2)
                Z3 = self.activation3.forward(A3)
                train_loss += self.activation3.log_loss(Z3, mini_y_train)
                y_pred = self.predict(Z3)
                y_true = np.argmax(mini_y_train, axis=1)
                accuracy_scores += len(y_pred[y_pred == y_true]) / len(mini_y_train)
                
                # Z3 = (20,10)
                dA3 = self.activation3.backward(mini_y_train) # 交差エントロピー誤差とソフトマックスを合わせている
                dZ2 = self.FC3.backward(dA3)
                dA2 = self.activation2.backward(dZ2)
                dZ1 = self.FC2.backward(dA2)
                dA1 = self.activation1.backward(dZ1)
                dZ0 = self.FC1.backward(dA1) # dZ0は使用しない
            
            train_loss / len(get_mini_batch)
            accuracy_scores = accuracy_scores / len(get_mini_batch)
            
            if self.verbose:
                print("train_loss{}, accuracy{}".format(train_loss, accuracy_scores))
                
    def predict(self, Z3):
           return np.argmax(Z3, axis=1)

In [750]:
# 全てのクラスを繋げる
class FC:    
    def __init__(self, n_nodes1, n_nodes2, initial, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        self.initial = initial
        self.W = self.initial.W(self.n_nodes1, self.n_nodes2)
        self.B = self.initial.B(self.n_nodes2)
        
        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        pass
    def forward(self, X, batch_size=20):
        
        # self.X = Z2, Z1,mini_X_trainを保存
        self.X = X
        A = X@self.W + self.B 
        return A
    
    def backward(self, dA):
        dZ = dA@self.W.T 
        self.dW = self.X.T@dA 
        self.dB = np.sum(dA, axis=0)
        
        self.optimizer.update(self) # update(self)のselfはこのクラス（FC）のインスタンスを渡したもの
        return dZ

class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, layer, lr=0.01):
        layer.W -= self.lr * layer.dW# (784, 20) (20,400) (784, 400)        
        layer.B -= self.lr * layer.dB # (400,)
        return 

# bais、重みの初期値
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
    
    def W(self, n_features, n_nodes1):
        W = self.sigma * np.random.randn(n_features, n_nodes1) #例１層目 (784, 400)
        return W
    
    def B(self, n_nodes1):
        B = self.sigma * np.random.randn(n_nodes1)
        return B
    
# シグモイド関数
class Sigmoid:
    def forward(self, X):
        self.s = 1/(1 + np.exp(-X))
        return self.s
    
    def backward(self, X):
        self.s_f = X * (1 - self.s) * self.s
        return self.s_f
      
# ソフトマックス    
class Softmax:
    def __init__(self, batch_size):
        self.batch_size = batch_size
    
    def forward(self, X):
        # オーバーフロー対策 
        X_max = np.max(X) # (20,10)
        self.sft = np.exp(X - X_max)/ np.sum(np.exp(X - X_max), axis=1, keepdims=True)
        return self.sft
    
    def backward(self, Y):
        self.batch_size = Y.shape[0]
        self.sft_f = self.sft - Y 
        return self.sft_f
    
    def log_loss(self, Z3, y):
        self.loss = (-1/self.batch_size) * np.sum(np.sum(y*np.log(Z3), axis=1)) # self.activation3.soft_max(A3)
        return self.loss
    
# ハイパポリックタンジェント
class Tanh:
    def __init__(self):
        self.t = None
        
    def forward(self, a):
        self.t = np.tanh(a)
        return self.t
    
    def backward(self, dz):
        self.t_f =  dz*(1 - self.t**2)
        return self.t_f

#　ReLU
class ReLU:
    def __init__(self):
        pass
    
    def forward(self, a):
        self.f = np.maximum(0, a)
        return self.f
    
    def backward(self, dZ):
        self.f_x = np.where(dZ > 0, 1, 0)
        return self.f_x

    
# sgmoid、tanhの初期の重み、バイアスはxavierが飽和しにくい
class XavierInitializer:
    def __init__(self):
        pass
    
    def W(self, n_features, n_nodes1):
        W =  np.random.randn(n_features, n_nodes1) / np.sqrt(n_nodes1) #例１層目 (784, 400)
        return W
    
    def B(self, n_nodes1):
        B = np.random.randn(n_nodes1) / np.sqrt(n_nodes1)
        return B

# reluの初期の重み、バイアスはheが飽和しにくい
class He:
    def __init__(self):
        pass
    
    def W(self, n_features, n_nodes1):
        W = np.random.randn(n_features, n_nodes1) / np.sqrt(2*n_nodes1) # 例１層目 (784, 400)
        return W

    def B(self, n_nodes1):
        B = np.random.randn(n_nodes1) / np.sqrt(2*n_nodes1)
        return B
        
# adagrad
class AdaGlad:
    def __init__(self, lr):
        self.lr = lr
        self.h_W = 1
        self.h_B = 1
        
    def update(self, layer):
        self.h_W +=  layer.dW**2
        self.h_B += layer.dB**2
        layer.W -= self.lr * 1/(np.sqrt(self.h_W)) * layer.dW
        layer.B -= self.lr * 1/(np.sqrt(self.h_B)) * layer.dB

# 【問題9】学習と推定

層の数や活性化関数を変えたいくつかのネットワークを作成してください。そして、MNISTのデータを学習・推定し、Accuracyを計算してください。

In [738]:
from keras.datasets import mnist
from sklearn.metrics import accuracy_score

# DL
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# 画像データを2次元に変換
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1,784)

# ワンホットに修正
y_train_hot = y_train.reshape(-1,1) == np.arange(10)
y_test_hot = y_test.reshape(-1,1) == np.arange(10)

# 0~!の数値に修正
X_train = X_train/255
X_test = X_test/255

In [739]:
 y_train.reshape(-1,1) == np.arange(10)

array([[False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False,  True, False]])

In [740]:
np.random.randn(2,3)/ np.sqrt(6)

array([[ 0.52943387,  0.54564183, -0.50442022],
       [ 0.44532683,  0.01008582, -0.65516497]])

In [741]:
y_pred = np.random.randint(1,3,10)
mini_y_train =np.random.randint(1,3,10)
print(y_pred)
print(mini_y_train)
y_pred[y_pred == mini_y_train]

[1 2 2 2 1 1 2 2 1 1]
[1 1 1 2 1 2 2 2 2 2]


array([1, 2, 1, 2, 2])

In [760]:
nn = ScratchDeepNeuralNetrowkClassifier(n_features=784, n_nodes1=400, n_nodes2=200, n_output=10, batch_size=20, sigma=0.01, lr=0.01, epoch=20,optimizer="ada",activation = "tanh",inital = "xavier",verbose=True)
nn.fit(X_train, y_train, X_test, y_test)

#sigmoid ◯
#softmax ◯
#relu ◯
#tanh ◯
# xavier ◯
# he ◯
# simple　◯

train_loss659.4967207980785, accuracy0.9388833333333314
train_loss278.73020152344094, accuracy0.9731499999999941
train_loss206.52246605024857, accuracy0.9810833333333271
train_loss165.91392845343154, accuracy0.9853333333333276
train_loss138.6810199358271, accuracy0.9884999999999945
train_loss118.82269499456542, accuracy0.9904666666666614
train_loss103.5787296765123, accuracy0.9921833333333286
train_loss91.46029831671515, accuracy0.9935166666666629
train_loss81.58268237363603, accuracy0.9946999999999968
train_loss73.38661357645425, accuracy0.9956499999999973
train_loss66.49495336629286, accuracy0.9965166666666647
train_loss60.6383622697975, accuracy0.9972499999999983
train_loss55.61518581732014, accuracy0.9978333333333322
train_loss51.26774053404802, accuracy0.998216666666666
train_loss47.47284589264382, accuracy0.9984166666666664
train_loss44.13419429572302, accuracy0.9986833333333329
train_loss41.17568286181387, accuracy0.9988166666666664
train_loss38.53696922321123, accuracy0.9989499